<a href="https://colab.research.google.com/github/Tessellate-Imaging/Monk_Object_Detection/blob/master/application_model_zoo/Example%20-%20Document%20Layout%20Analysis%20(SSD512).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Document Layout Analysis Using SSD

## About the network:
1. Paper on SSD: https://arxiv.org/abs/1512.02325

2. Blog-1 on SSD: https://towardsdatascience.com/review-ssd-single-shot-detector-object-detection-851a94607d11

3. Blog-2 on SSD: https://medium.com/@jonathan_hui/ssd-object-detection-single-shot-multibox-detector-for-real-time-processing-9bd8deac0e06

# Table of Contents

### 1. Installation Instructions
### 2. Use trained Model for Document Layout Analysis
### 3. How to train using PRImA Layout Analysis Dataset

## MXRCNN pipeline of Monk Object Detection Library has been used for implementing this model. 

 - After some comparisons, it was found out that VGG16 performs better than ResNet101 for object detection task using FasterRCNN, so VGG16 is chosen for the backend. Firstly, the dataset was converted from Monk Format to COCO Format. 
 - The model was trained for 3 epochs with learning rate of 0.005 and then for 3 more epochs with learning rate of 0.001. The image was preprocessed to smaller size (min 300px, max 500px) and normalsied using mean and standard deviation calculated in preprocessing notebook. 
 - Batch size has been kept at 2 because more than that was causing CUDAOutOfMemory error. It achieved RPN Accuracy=0.807714 and RCNN Accuracy= 0.750662.

# Installation

- Run these commands

    - git clone https://github.com/Tessellate-Imaging/Monk_Object_Detection.git

    - cd Monk_Object_Detection/1_gluoncv_finetune/installation

- Select the right requirements file and run

    - cat requirements_cuda10.1.txt | xargs -n 1 -L 1 pip install

In [ ]:
! git clone https://github.com/Tessellate-Imaging/Monk_Object_Detection.git

In [ ]:
# For colab use the command below
#! cd Monk_Object_Detection/1_gluoncv_finetune/installation && cat requirements_colab.txt | xargs -n 1 -L 1 pip install


# For Local systems and cloud select the right CUDA version
!cd Monk_Object_Detection/1_gluoncv_finetune/installation && cat requirements_cuda10.1.txt | xargs -n 1 -L 1 pip install

# Use Already Trained Model for Demo

In [ ]:
import os
import sys
sys.path.append("Monk_Object_Detection/1_gluoncv_finetune/lib/");

In [ ]:
from inference_prototype import Infer

In [ ]:
#Download trained model

In [ ]:
! wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1E6T7RKGwy-v1MUxVJm-rxt5XcRyr2SQ7' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1E6T7RKGwy-v1MUxVJm-rxt5XcRyr2SQ7" -O obj_dla_ssd512_trained.zip && rm -rf /tmp/cookies.txt

In [ ]:
! unzip -qq obj_dla_ssd512_trained.zip

In [ ]:
model_name = "ssd_512_vgg16_atrous_coco";
params_file = "dla_ssd512/dla_ssd512-vgg16.params";
class_list = ["paragraph", "heading", "credit", "footer", "drop-capital", "floating", "noise", "maths", "header", "caption", "image", "linedrawing", "graphics", "fname", "page-number", "chart", "separator", "table"];

In [ ]:
gtf = Infer(model_name, params_file, class_list, use_gpu=True);

In [ ]:
# download test images

In [ ]:
! wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1VEkfJuicIr-STIqYOI-UruDDttWGXNxw' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1VEkfJuicIr-STIqYOI-UruDDttWGXNxw" -O Test_Images.zip && rm -rf /tmp/cookies.txt

In [ ]:
! unzip -qq Test_Images.zip

In [ ]:
img_name = "Test_Images/test1.jpg"; 
visualize = True;
thresh = 0.3;
output = gtf.run(img_name, visualize=visualize, thresh=thresh);

In [ ]:
img_name = "Test_Images/test2.jpg"; 
visualize = True;
thresh = 0.3;
output = gtf.run(img_name, visualize=visualize, thresh=thresh);

In [ ]:
img_name = "Test_Images/test3.jpg"; 
visualize = True;
thresh = 0.4;
output = gtf.run(img_name, visualize=visualize, thresh=thresh);

# Train Your Own Model

## Dataset Credits
- https://www.primaresearch.org/datasets/Layout_Analysis

In [ ]:
#Download Dataset

In [ ]:
! wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1iBfafT1WHAtKAW0a1ifLzvW5f0ytm2i_' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1iBfafT1WHAtKAW0a1ifLzvW5f0ytm2i_" -O PRImA_Layout_Analysis_Dataset.zip && rm -rf /tmp/cookies.txt

In [ ]:
! unzip -qq PRImA_Layout_Analysis_Dataset.zip

# Data Preprocessing

### Library for Data Augmentation
Refer to https://github.com/albumentations-team/albumentations for more details

### Data Preprocessing

  - Normalisation: Calculated Mean & Standard deviation of training images (3 images taken out of dataset for inference) to feed into model for normalisation (used in FasterRCNN).

  - Format Conversion: TIFF format was causing problems in data augmentation and training on TIFF images was more than 5x slower than JPEG format images because of their huge size. Therefore, TIFF images were converted to JPEG format images.

  - Selective Data Augmentation: In the raw dataset 4750+ paragraph type objects and only 10-30 frames, graphics, etc. type objects which led to huge bias in dataset. To generate more data and to decrease bias, a customised function has been implemented from the scratch. This function produces random translational augmentated images of only those images which have minority classes in them. Using this function, the dataset size increased from 475 images to 1783 images. If data augmentation has been done on every image, there would have been 24000+ paragraphs in the dataset whereas there are 19568 now, which slightly improves the bias. (Exact numbers are in the data preprocessing notebook). The augmented images had to be saved in the dataset because augmentation function couldn't be called on the go.

  - Conversion to VOC to Monk type- Coversion so that Monk format can be later used in SSD Model, converted to yolo type for Yolo model, and COCO format for FasterRCNN Model.

In [ ]:
! pip install albumentations

In [ ]:
import os
import sys
import cv2
import numpy as np
import pandas as pd
from PIL import Image
import albumentations as A
import glob
import matplotlib.pyplot as plt
import xmltodict
import json
from tqdm.notebook import tqdm
from pycocotools.coco import COCO

In [ ]:
root_dir = "PRImA Layout Analysis Dataset/";
img_dir = "Images/";
anno_dir = "XML/";
final_root_dir="Document_Layout_Analysis/" #Directory for jpeg and augmented images

In [ ]:
if not os.path.exists(final_root_dir):
    os.makedirs(final_root_dir)

if not os.path.exists(final_root_dir+img_dir):
    os.makedirs(final_root_dir+img_dir)

## TIFF Image Format to JPEG Image Format

In [ ]:
for name in glob.glob(root_dir+img_dir+'*.tif'):
    im = Image.open(name)
    name = str(name).rstrip(".tif")
    name = str(name).lstrip(root_dir)
    name = str(name).lstrip(img_dir)
    im.save(final_root_dir+ img_dir+ name + '.jpg', 'JPEG')

# Format Conversion and Data Augmentation

As most part of a document is text, there were far more paragraphs in the dataset than there were other labels such as tables or graphs. To handle this huge bias in the dataset, we augmented only those document images which had one of these minority labels in them. For example, if the document only had paragraphs and images, then we didn’t augment it. But if it had tables, charts, graphs or any other minority label, we augmented that image by many folds. This process helped in reducing the bias in the dataset by around 25%. This selection and augmentation has been done during the format conversion from VOC to Monk Format.

## Given format- VOC Format

### Dataset Directory Structure

    ./PRImA Layout Analysis Dataset/ (root_dir)
          |
          |-----------Images (img_dir)
          |              |
          |              |------------------img1.jpg
          |              |------------------img2.jpg
          |              |------------------.........(and so on)
          |
          |
          |-----------Annotations (anno_dir)
          |              |
          |              |------------------img1.xml
          |              |------------------img2.xml
          |              |------------------.........(and so on)
          


## Required Format- Monk Format

### Dataset Directory Structure

    ./Document_Layout_Analysis/ (final_root_dir)
          |
          |-----------Images (img_dir)
          |              |
          |              |------------------img1.jpg
          |              |------------------img2.jpg
          |              |------------------.........(and so on)
          |
          |
          |-----------train_labels.csv (anno_file)
          
          
### Annotation file format

           | Id         | Labels                                 |
           | img1.jpg   | x1 y1 x2 y2 label1 x1 y1 x2 y2 label2  |
           
- Labels:  xmin ymin xmax ymax label
- xmin, ymin - top left corner of bounding box
- xmax, ymax - bottom right corner of bounding box

In [ ]:
files = os.listdir(root_dir + anno_dir);

In [ ]:
combined = [];

### Data Augmentation Function

In [ ]:
def augmentData(fname, boxes):
    image = cv2.imread(final_root_dir+img_dir+fname)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
 
    transform = A.Compose([
        A.IAAPerspective(p=0.7),   
        A.ShiftScaleRotate(shift_limit=0.1, scale_limit=0.1, rotate_limit=5, p=0.5),
        A.IAAAdditiveGaussianNoise(),
        A.ChannelShuffle(),
        A.RandomBrightnessContrast(),
        A.RGBShift(p=0.8),
        A.HueSaturationValue(p=0.8)
        ], bbox_params=A.BboxParams(format='pascal_voc', min_visibility=0.2))
    
    for i in range(1, 9):
        label=""
        transformed = transform(image=image, bboxes=boxes)
        transformed_image = transformed['image']
        transformed_bboxes = transformed['bboxes']
        #print(transformed_bboxes)
        flag=False
        for box in transformed_bboxes:
            x_min, y_min, x_max, y_max, class_name = box
            if(xmax<=xmin or ymax<=ymin):
                flag=True
                break
            label+= str(int(x_min))+' '+str(int(y_min))+' '+str(int(x_max))+' '+str(int(y_max))+' '+class_name+' '
                        
        if(flag):
            continue
        cv2.imwrite(final_root_dir+img_dir+str(i)+fname, transformed_image)
        label=label[:-1]
        combined.append([str(i) + fname, label])


# VOC to Monk Format Conversion
Applying Data Augmentation only on those images which contain atleast 1 minority class so as to reduce bias in the dataset

In [ ]:
#label generation for csv
for i in tqdm(range(len(files))):
    box=[];
    augment=False;
    annoFile = root_dir + anno_dir + files[i];
    f = open(annoFile, 'r');
    my_xml = f.read();
    anno= dict(dict(dict(xmltodict.parse(my_xml))['PcGts'])['Page'])
    fname=""
    for j in range(len(files[i])):
        if((files[i][j])>='0' and files[i][j]<='9'):
            fname+=files[i][j];
    fname+=".jpg"
    image = cv2.imread(final_root_dir+img_dir+fname)
    height, width = image.shape[:2]    
    label_str = ""
    for key in anno.keys():
        if(key=='@imageFilename' or key=='@imageWidth' or key=='@imageHeight'):
            continue
        if(key=="TextRegion"):
            if(type(anno["TextRegion"]) == list):
                for j in range(len(anno["TextRegion"])):
                    text=anno["TextRegion"][j]
                    xmin=width
                    ymin=height
                    xmax=0
                    ymax=0
                    if(text["Coords"]):
                        if(text["Coords"]["Point"]):
                            for k in range(len(text["Coords"]["Point"])):
                                coordinates=anno["TextRegion"][j]["Coords"]["Point"][k]
                                xmin= min(xmin, int(coordinates['@x']));
                                ymin= min(ymin, int(coordinates['@y']));
                                xmax= min(max(xmax, int(coordinates['@x'])), width);
                                ymax= min(max(ymax, int(coordinates['@y'])), height);
                            if('@type' in text.keys()):    
                                label_str+= str(xmin)+' '+str(ymin)+' '+str(xmax)+' '+str(ymax)+' '+text['@type']+' '
                                if(xmax<=xmin or ymax<=ymin):
                                    continue
                                tbox=[];
                                tbox.append(xmin)
                                tbox.append(ymin)
                                tbox.append(xmax)
                                tbox.append(ymax)
                                tbox.append(text['@type'])
                                box.append(tbox)
            else:
                text=anno["TextRegion"]
                xmin=width
                ymin=height
                xmax=0
                ymax=0
                if(text["Coords"]):
                    if(text["Coords"]["Point"]):
                        for k in range(len(text["Coords"]["Point"])):
                            coordinates=anno["TextRegion"]["Coords"]["Point"][k]
                            xmin= min(xmin, int(coordinates['@x']));
                            ymin= min(ymin, int(coordinates['@y']));
                            xmax= min(max(xmax, int(coordinates['@x'])), width);
                            ymax= min(max(ymax, int(coordinates['@y'])), height);
                        if('@type' in text.keys()):    
                            label_str+= str(xmin)+' '+str(ymin)+' '+str(xmax)+' '+str(ymax)+' '+text['@type']+' '
                            if(xmax<=xmin or ymax<=ymin):
                                continue
                            tbox=[];
                            tbox.append(xmin)
                            tbox.append(ymin)
                            tbox.append(xmax)
                            tbox.append(ymax)
                            tbox.append(text['@type'])
                            box.append(tbox)
        
        else:
            val=""
            if(key=='GraphicRegion'):
                val="graphics"
                augment=True
            elif(key=='ImageRegion'):
                val="image"
            elif(key=='NoiseRegion'):
                val="noise"
                augment=True
            elif(key=='ChartRegion'):
                val="chart"
                augment=True
            elif(key=='TableRegion'):
                val="table"
                augment=True
            elif(key=='SeparatorRegion'):
                val="separator"
            elif(key=='MathsRegion'):
                val="maths"
                augment=True
            elif(key=='LineDrawingRegion'):
                val="linedrawing"
                augment=True
            else:
                val="frame"
                augment=True

            
            if(type(anno[key]) == list):
                for j in range(len(anno[key])):
                    text=anno[key][j]
                    xmin=width
                    ymin=height
                    xmax=0
                    ymax=0
                    if(text["Coords"]):
                        if(text["Coords"]["Point"]):
                            for k in range(len(text["Coords"]["Point"])):
                                coordinates=anno[key][j]["Coords"]["Point"][k]
                                xmin= min(xmin, int(coordinates['@x']));
                                ymin= min(ymin, int(coordinates['@y']));
                                xmax= min(max(xmax, int(coordinates['@x'])), width);
                                ymax= min(max(ymax, int(coordinates['@y'])), height);
                        label_str+= str(xmin)+' '+str(ymin)+' '+str(xmax)+' '+str(ymax)+' '+ val +' '
                        if(xmax<=xmin or ymax<=ymin):
                            continue
                        tbox=[];
                        tbox.append(xmin)
                        tbox.append(ymin)
                        tbox.append(xmax)
                        tbox.append(ymax)
                        tbox.append(val)
                        box.append(tbox)
            else:
                text=anno[key]
                xmin=width
                ymin=height
                xmax=0
                ymax=0
                if(text["Coords"]):
                    if(text["Coords"]["Point"]):
                        for k in range(len(text["Coords"]["Point"])):
                            coordinates=anno[key]["Coords"]["Point"][k]
                            xmin= min(xmin, int(coordinates['@x']));
                            ymin= min(ymin, int(coordinates['@y']));
                            xmax= min(max(xmax, int(coordinates['@x'])), width);
                            ymax= min(max(ymax, int(coordinates['@y'])), height);  
                        label_str+= str(xmin)+' '+str(ymin)+' '+str(xmax)+' '+str(ymax)+' '+val+' '
                        if(xmax<=xmin or ymax<=ymin):
                            continue
                        tbox=[];
                        tbox.append(xmin)
                        tbox.append(ymin)
                        tbox.append(xmax)
                        tbox.append(ymax)
                        tbox.append(val)
                        box.append(tbox)

    label_str=label_str[:-1]
    combined.append([fname, label_str])

    if(augment):
        augmentData(fname, box)
        

In [ ]:
df = pd.DataFrame(combined, columns = ['ID', 'Label']);
df.to_csv(final_root_dir + "/train_labels.csv", index=False);

# Training

In [ ]:
import os
import sys

sys.path.append("Monk_Object_Detection/1_gluoncv_finetune/lib/");

In [ ]:
from detector_prototype import Detector

In [ ]:
gtf = Detector();

In [ ]:
root = "Document_Layout_Analysis/";
img_dir = "Images/";
anno_file = "train_labels.csv";
batch_size=8;

In [ ]:
gtf.Dataset(root, img_dir, anno_file, batch_size=batch_size);

### Available models
    ssd_300_vgg16_atrous_coco
    ssd_300_vgg16_atrous_voc
    ssd_512_vgg16_atrous_coco
    ssd_512_vgg16_atrous_voc
    ssd_512_resnet50_v1_coco
    ssd_512_resnet50_v1_voc
    ssd_512_mobilenet1.0_voc
    ssd_512_mobilenet1.0_coco
    yolo3_darknet53_voc
    yolo3_darknet53_coco
    yolo3_mobilenet1.0_voc
    yolo3_mobilenet1.0_coco

In [ ]:
#vgg16 architecture, with atrous convolutions, pretrained on COCO dataset is used for this task
pretrained = True;         
gpu=True;
model_name = "ssd_512_vgg16_atrous_coco";

In [ ]:
gtf.Model(model_name, use_pretrained=pretrained, use_gpu=gpu);

In [ ]:
gtf.Set_Learning_Rate(0.003);

In [ ]:
epochs=30;
params_file = "saved_model.params";

In [ ]:
gtf.Train(epochs, params_file);

# Inference

In [ ]:
import os
import sys
sys.path.append("Monk_Object_Detection/1_gluoncv_finetune/lib/");

In [ ]:
from inference_prototype import Infer

In [ ]:
model_name = "ssd_512_vgg16_atrous_coco";
params_file = "saved_model.params";
class_list = ["paragraph", "heading", "credit", "footer", "drop-capital", "floating", "noise", "maths", "header", "caption", "image", "linedrawing", "graphics", "fname", "page-number", "chart", "separator", "table"];

In [ ]:
gtf = Infer(model_name, params_file, class_list, use_gpu=True);

In [ ]:
img_name = "Test_Images/test1.jpg"; 
visualize = True;
thresh = 0.3;
output = gtf.run(img_name, visualize=visualize, thresh=thresh);

In [ ]:
img_name = "Test_Images/test2.jpg"; 
visualize = True;
thresh = 0.3;
output = gtf.run(img_name, visualize=visualize, thresh=thresh);

In [ ]:
img_name = "Test_Images/test3.jpg"; 
visualize = True;
thresh = 0.4;
output = gtf.run(img_name, visualize=visualize, thresh=thresh);

### Inference
SSD512 produces outputs with very high confidence, a lot of them being 0.9+. It was also the only model which was able to identify footer and noises like division lines in the document. But it was also producing repetitive or incorrect headings such as ‘floating’ in the 2nd example (extra box with incorrect label), and graphics and paragraph in the third (2 boxes with different labels for the same region).

If these small details like footer, separator, etc. are crucial for your work and the focus is more on bounding box prediction than classification, go for SSD512. It should also be considered that gluoncv-finetune pipeline of Monk AI (which has been used for SSD512) also provides architectures which are pre-trained on various other datasets, such as COCO dataset.